In [9]:
import os
import cv2
import numpy as np

def extract_face(image, haar_cascade_path):
    # Haar Cascade 로드
    face_cascade = cv2.CascadeClassifier(haar_cascade_path)
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.02, minNeighbors=2)
    
    # 얼굴이 발견되지 않으면 None 반환
    if len(faces) == 0:
        return None, None
    
    # 첫 번째 얼굴 추출
    (x, y, w, h) = faces[0]
    face = image[y:y+h, x:x+w]
    return face, (x, y, w, h)

def blend_faces(base_image, face_image, position):
    (x, y, w, h) = position
    
    # 얼굴 이미지 크기 조정
    face_resized = cv2.resize(face_image, (w, h))
    
    # ROI 설정
    roi = base_image[y:y+h, x:x+w]
    
    # 얼굴 이미지 알파 블렌딩
    blended = cv2.addWeighted(roi, 0.3, face_resized, 0.7, 0)
    
    # 원래 이미지에 합성된 얼굴 붙여넣기
    base_image[y:y+h, x:x+w] = blended
    return base_image

def main(user_face_path, animal_image_path, output_path):
    # 파일 경로 확인
    if not os.path.exists(user_face_path):
        print(f"사용자 얼굴 이미지 파일이 존재하지 않습니다: {user_face_path}")
        return
    if not os.path.exists(animal_image_path):
        print(f"동물 이미지 파일이 존재하지 않습니다: {animal_image_path}")
        return
    
    # 사용자 얼굴 이미지 로드
    user_image = cv2.imread(user_face_path)
    if user_image is None:
        print(f"사용자 얼굴 이미지를 로드할 수 없습니다: {user_face_path}")
        return
    
    user_face_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    user_face, user_position = extract_face(user_image, user_face_cascade_path)
    if user_face is None:
        print("사용자 얼굴을 찾을 수 없습니다.")
        return
    
    # 동물 이미지 로드
    animal_image = cv2.imread(animal_image_path)
    if animal_image is None:
        print(f"동물 이미지를 로드할 수 없습니다: {animal_image_path}")
        return
    
    animal_face_cascade_path = 'haarcascade_frontalcatface.xml'
    animal_face, animal_position = extract_face(animal_image, animal_face_cascade_path)
    if animal_face is None:
        print("동물 얼굴을 찾을 수 없습니다.")
        return
    
    # 동물 이미지에 사용자 얼굴 합성
    result_image = blend_faces(animal_image, user_face, animal_position)
    
    # 결과 이미지 저장
    cv2.imwrite(output_path, result_image)
    print(f"합성된 이미지를 {output_path}에 저장했습니다.")

# 경로 설정
user_face_path = 'winter.jpg'
animal_image_path = 'ragdoll.jpg'
output_path = 'winterragdoll.jpg'

main(user_face_path, animal_image_path, output_path)


합성된 이미지를 winterragdoll.jpg에 저장했습니다.


In [1]:
import os
import cv2
import numpy as np
import dlib

def extract_face(image):
    # dlib의 얼굴 탐지기 로드
    detector = dlib.get_frontal_face_detector()
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    # 얼굴이 발견되지 않으면 None 반환
    if len(faces) == 0:
        return None, None
    
    # 첫 번째 얼굴 추출
    face = faces[0]
    x, y, w, h = face.left(), face.top(), face.width(), face.height()
    face_image = image[y:y+h, x:x+w]
    return face_image, (x, y, w, h)

def blend_faces(base_image, face_image, position):
    (x, y, w, h) = position
    
    # 얼굴 이미지 크기 조정
    face_resized = cv2.resize(face_image, (w, h))
    
    # ROI 설정
    roi = base_image[y:y+h, x:x+w]
    
    # 얼굴 이미지 알파 블렌딩
    blended = cv2.addWeighted(roi, 0.5, face_resized, 0.5, 0)
    
    # 원래 이미지에 합성된 얼굴 붙여넣기
    base_image[y:y+h, x:x+w] = blended
    return base_image

def main(user_face_path, animal_image_path, output_path):
    # 파일 경로 확인
    if not os.path.exists(user_face_path):
        print(f"사용자 얼굴 이미지 파일이 존재하지 않습니다: {user_face_path}")
        return
    if not os.path.exists(animal_image_path):
        print(f"동물 이미지 파일이 존재하지 않습니다: {animal_image_path}")
        return
    
    # 사용자 얼굴 이미지 로드
    user_image = cv2.imread(user_face_path)
    if user_image is None:
        print(f"사용자 얼굴 이미지를 로드할 수 없습니다: {user_face_path}")
        return
    
    user_face, user_position = extract_face(user_image)
    if user_face is None:
        print("사용자 얼굴을 찾을 수 없습니다.")
        return
    
    # 동물 이미지 로드
    animal_image = cv2.imread(animal_image_path)
    if animal_image is None:
        print(f"동물 이미지를 로드할 수 없습니다: {animal_image_path}")
        return
    
    animal_face, animal_position = extract_face(animal_image)
    if animal_face is None:
        print("동물 얼굴을 찾을 수 없습니다.")
        return
    
    # 동물 이미지에 사용자 얼굴 합성
    result_image = blend_faces(animal_image, user_face, animal_position)
    
    # 결과 이미지 저장
    cv2.imwrite(output_path, result_image)
    print(f"합성된 이미지를 {output_path}에 저장했습니다.")

# 경로 설정
user_face_path = 'karina.jpg'
animal_image_path = 'ragdoll.jpg'
output_path = 'karinaragdoll.jpg'

main(user_face_path, animal_image_path, output_path)


동물 얼굴을 찾을 수 없습니다.
